# System Demo
- Paper: Towards interactive robotics: Zero-shot action sequence generation using reasoning through ChatGPT
- This system presents a take on a ReAct model (more info: https://react-lm.github.io/)
- Creator: Andrei Dragomir

Add "fancy" version of the system demo here.
pip install tts
follow this guide to plug in text-to-speech https://www.youtube.com/watch?v=MYRgWwis1Jk
!! call tts with gpu=True to use gpu


## Imports and initialization

In [ ]:
import os
import tiktoken
import time
from gpt_controller.config import *
from gpt_controller.util.models import *
from gpt_controller.util.labels import *
from gpt_controller.cognition.machine import Machine

machine = Machine(None)

## Testing chatGPT

#### Benchmarks

In [ ]:
# Function to get prompt deck from the prompt directory as a dictionary
# Returns a dictionary of prompt names and prompts
def get_prompt_deck() -> dict[str, str]:
    prompt_deck = {}
    for root, dirs, files in os.walk(PROMPT_PATH):
        for name in files:
            if name.endswith(".txt"):
                prompt_location = os.path.abspath(os.path.join(root, name))
                try:
                    with open(prompt_location, "r") as f:
                        prompt = f.read()
                        f.flush()
                    prompt_deck[name] = prompt
                except OSError as e:
                    print("Error: Prompt {} could not be loaded with reason: {}".format(name, e.args[0]))
                    continue
    return prompt_deck    
        
def num_tokens_prompt(prompt: str):
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.encoding_for_model(CHATGPT_MODEL)
    num_tokens = len(encoding.encode(prompt))
    return num_tokens

In [ ]:
for name, prompt in get_prompt_deck().items():
    num_tokens = num_tokens_prompt(prompt)
    if num_tokens == 0:
        continue
    print("Prompt: {} \nNumber of tokens: {}\n".format(name, num_tokens))

#### Targeted prompt tests

- This section provides tests for specific prompts that are used in the system.
- The tests are done by running the prompt through chatGPT and checking if the output is the expected one.

##### Labelling

- You can make sets of prompts for a specific type of label and run them all at once using the helper function `run_label_tests()`.
    - You must pass a list of strings that you want to test
    - You must specify what label you want to test.
    - The function returns an accuracy score of the tests.

In [ ]:
def run_label_tests(prompt_deck : list[tuple[Label, str]], expected_label : Label):
    right_answers = 0
    for label, prompt in prompt_deck:
        provided_label = machine.label(prompt, expected_label.__class__)
        time.sleep(2)
        if provided_label == label:
            right_answers += 1
        else:
            print("Prompt: {}\nExpected Label: {}\nProvided Label: {}\n".format(prompt, expected_label, provided_label))
    return right_answers/len(prompt_deck)

Task type labelling

In [ ]:
task_set = [
    "Peel 5 potatoes.",
    "Stir the pot of soup.",
    "Chop onions.",
    "Pour ingredients into a mixing bowl.",
    "Wash fruits and vegetables.",
    "Measure ingredients for the recipe.",
    "Open the jar of jam.",
    "Grate cheese.",
    "Preheat the oven to a specific temperature.",
    "Set a timer for cooking or baking.",
    "Mix ingredients together in a bowl.",
    "Slice some bread.",
    "Pour liquids into containers.",
    "Clean countertop with the sponge.",
    "Load the dishwasher.",
    "Retrieve items from cabinets or shelves.",
    "Remove hot dishes from the oven.",
    "Garnish plates with herbs or sauces.",
    "Arrange food on a serving platter.",
    "Brew a cup of coffee or tea.",
    "Clean kitchen utensils.",
    "Wipe spills from the floor.",
    "Refill water bottles or containers.",
    "Organize pantry items.",
    "Dispose of food waste."
]


labelled_task_set = [(UserInputLabel.TASK, prompt) for prompt in task_set]

run_label_tests(labelled_task_set, UserInputLabel.TASK)

Generated task labelling

In [ ]:
task_topic = "Make a salad"

task_sequence = [
    "Recall any knowledge about the location of the ingredients.",
    "Move to the refrigerator.",
    "Open the fridge",
    "Look for vegetables for the salad.",
    "Pick up a tomato",
    "Place it on the counter.",
    "Pick up a cucumber.",
    "Place it on the counter.",
    "Pick up a lettuce.",
    "Place it on the counter.",
    "Close the refrigerator.",
    "Move to the counter.",
    "Find a knife.",
    "Ask where the knife could be located.",
    "Pick up the knife.",
    "Slice the vegetables.",
    "Which vegetable should be sliced first?",
    "Slice the tomato.",
    "Slice the cucumber.",
    "Move to the sink."
]

for prompt in task_sequence:
    label = machine.label(prompt, TaskLabel)
    print("Prompt: {}\nLabel: {}\n".format(prompt, label))
    time.sleep(2)


##### User input segmentation

In [ ]:
input_string = '''Tell me what color the fork has then tell me what color the knife has. You should only use your cognitive abilities, without perceiving the environment to extract data'''

conversation = Conversation(ConversationType.LABELLING)
conversation.messages.append(Message(Role.SYSTEM, machine.load_prompt('segment_input.txt')))
conversation.messages.append(Message(Role.USER, input_string))

completion = machine.process(conversation.messages)
print(completion['content'])



## Future Work

- Implementing a ReAct system for interfacing with chatGPT
    - ReAct is a system that allows for the generation of actions from a given state.
    - It is a system that is trained on a dataset of actions and states.

    **Inspiration**
    https://react-lm.github.io/

- Implement defensive json parsing for the chatGPT system
    - Use a json schema to validate the input json
    - Use a library like `Langchain` or `llmparser` to recover from slightly malformed json outputs